<a href="https://colab.research.google.com/github/azrlmaster/sleeper-projections/blob/main/Sleeper_API_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
from collections import defaultdict

# --- Configuration ---
# Set your Sleeper username here
SLEEPER_USERNAME = "thebmarv"
# Set the season you want to pull data for
LEAGUE_YEAR = "2025"

# --- API Keys & URLs ---
# You would need to get an API key from a service like The Odds API
# and put it here.
THE_ODDS_API_KEY = "your_odds_api_key_here"
ODDS_API_BASE_URL = "https://api.the-odds-api.com/v4"

API_BASE_URL = "https://api.sleeper.app/v1"


def get_user(username: str):
    """Fetches a user's information using their username."""
    print(f"Attempting to fetch user ID for '{username}'...")
    url = f"{API_BASE_URL}/user/{username}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        user_data = response.json()
        if not user_data or "user_id" not in user_data:
            print(f"Error: Could not find user '{username}'. Check if the username is correct.")
            return None
        print(f"Successfully found User ID: {user_data['user_id']}\n")
        return user_data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching user data: {e}")
        return None

def get_leagues_for_user(user_id: str, year: str):
    """Fetches all of a user's leagues for a specific year."""
    print(f"Attempting to fetch leagues for user {user_id} for the {year} season...")
    url = f"{API_BASE_URL}/user/{user_id}/leagues/nfl/{year}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        leagues_data = response.json()
        if not leagues_data:
            print(f"No leagues found for the {year} season.")
            return []
        print(f"Successfully found {len(leagues_data)} leagues.\n")
        return leagues_data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching league data: {e}")
        return []

def get_all_nfl_players():
    """Fetches the master list of all NFL players from Sleeper."""
    print("Fetching master list of all NFL players (this may take a moment)...")
    url = f"{API_BASE_URL}/players/nfl"
    try:
        response = requests.get(url)
        response.raise_for_status()
        print("Successfully fetched player data.\n")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching the player master list: {e}")
        return None

def get_league_rosters(league_id: str):
    """Fetches all rosters for a given league."""
    url = f"{API_BASE_URL}/league/{league_id}/rosters"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching rosters for league {league_id}: {e}")
        return None

def get_projections_from_odds_api(player_name: str):
    """
    *** BLUEPRINT FUNCTION ***
    This function would fetch player prop odds from your chosen API.
    You will need to implement the actual API call and parsing here.
    """
    # Example: you would make a call to The Odds API for a specific player
    # url = f"{ODDS_API_BASE_URL}/sports/americanfootball_nfl/events?apiKey={THE_ODDS_API_KEY}"
    # The API response would need to be parsed to find the prop bets for the given player.

    # This function should return a dictionary of the prop values, e.g.:
    # {
    #   "pass_yd": 250.5,
    #   "pass_td": 1.5,
    #   "rush_yd": 10.0,
    #   "rec_yd": 0.0,
    #   "rec": 0.0,
    #   "rec_td": 0.0,
    #   "rush_td": 0.1
    # }
    # For now, it returns None. You must implement this part.
    print(f"  - (Blueprint) Pretending to fetch odds for {player_name}...")
    return None

def calculate_fantasy_points(player_props: dict, scoring_settings: dict):
    """Calculates a fantasy point total from props and league scoring rules."""
    if not player_props or not scoring_settings:
        return 0.0

    total_points = 0.0

    # Example calculation logic. You would need to expand this for all scoring types.
    # This requires matching the stat name from your odds API to Sleeper's scoring settings.
    total_points += player_props.get("pass_yd", 0.0) * scoring_settings.get("pass_yd", 0.0)
    total_points += player_props.get("pass_td", 0.0) * scoring_settings.get("pass_td", 0.0)
    total_points += player_props.get("rec", 0.0) * scoring_settings.get("rec", 0.0)
    # ... and so on for all other categories (rushing, fumbles, etc.)

    return total_points

def main():
    """Main function to run the script."""
    if SLEEPER_USERNAME == "your_username_here":
        print("Please update the SLEEPER_USERNAME variable with your Sleeper username.")
        return
    if THE_ODDS_API_KEY == "your_odds_api_key_here":
        print("Please sign up for a sports odds API and add your key to the THE_ODDS_API_KEY variable.")
        return

    # 1. Fetch all the necessary data from APIs
    user = get_user(SLEEPER_USERNAME)
    if not user: return

    all_players = get_all_nfl_players()
    if not all_players: return

    leagues = get_leagues_for_user(user["user_id"], LEAGUE_YEAR)
    if not leagues: return

    # 2. Process the data for each league
    for league in leagues:
        print(f"\nLeague: {league['name']}")

        # We need the specific scoring settings for this league
        scoring_settings = league.get('scoring_settings', {})

        rosters = get_league_rosters(league['league_id'])
        if not rosters: continue

        my_roster = next((r for r in rosters if r.get('owner_id') == user['user_id']), None)

        if my_roster and my_roster.get('players'):
            total_projection = 0.0
            print("Processing your roster:")
            for player_id in my_roster['players']:
                player_info = all_players.get(player_id)
                if player_info:
                    player_name = player_info.get('full_name', 'N/A')

                    # Step 1: Get prop bet lines for the player
                    player_props = get_projections_from_odds_api(player_name)

                    # Step 2: Calculate fantasy points based on this league's rules
                    player_projection = calculate_fantasy_points(player_props, scoring_settings)
                    total_projection += player_projection

                    position = player_info.get('position', 'N/A')
                    team = player_info.get('team', 'N/A')
                    # Because our blueprint function returns None, this will show 0.00 for now.
                    print(f"    - {player_name} ({position}, {team}): {player_projection:.2f} points")

            print(f"  ---------------------------------")
            print(f"  Total Team Projection: {total_projection:.2f} points")
        else:
            print("  Could not find your roster in this league.")

if __name__ == "__main__":
    main()